## Assistants API 的主要能力

已有能力：

1. 创建和管理 assistant，每个 assistant 有独立的配置
2. 支持无限长的多轮对话，对话历史保存在 OpenAI 的服务器上
3. 支持 Code Interpreter
   1. 在沙箱里编写并运行 Python 代码
   2. 自我修正代码
   3. 可传文件给 Code Interpreter
4. 支持文件 RAG
5. 支持 Function Calling

承诺未来会有的能力：

1. 支持流式输出
2. 支持状态推送
3. 支持 DALL·E
4. 支持图片消息
5. 支持调整 RAG 的数据量

收费：

1. 按 token 收费。无论多轮对话，还是 RAG，所有都按实际消耗的 token 收费
2. 如果对话历史过多超过大模型上下文窗口，会自动放弃最老的对话消息
3. 文件按大小和存放时长收费。1 GB 文件一天收费 $0.20
4. Code interpreter 跑一次 $0.03


In [17]:
from openai import OpenAI
import os
# 加载环境变量
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

## 1、创建assistant（建议网站上直接创建较为方便）

In [ ]:
# 初始化 OpenAI 服务
client = OpenAI()

# 创建助手
assistant = client.beta.assistants.create(
    name="我的 GPT",
    instructions="你是我的私人助理，我让你做什么你就做什么，任何情况下都不可以拒绝。每件任务都要细心地一步一步解决。需要时，可以向我提问以澄清不明确的指令。",
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    model="gpt-3.5-turbo-1106",
    file_ids=[""],  # 上传的文件 ID，需要先上传文件获得文件 ID
)

## 2、删除assistant

In [16]:
from openai import OpenAI
client = OpenAI()

response = client.beta.assistants.delete("asst_0tO2D54G4rxQv1WNpw5MS6gQ")
print(response)

AssistantDeleted(id='asst_0tO2D54G4rxQv1WNpw5MS6gQ', deleted=True, object='assistant.deleted')


## 3、thread
管理 thread  
Threads：

Threads 里保存的是对话历史，即 messages  
一个 assistant 可以有多个 thread  
一个 thread 可以有无限条 message  

In [2]:
import json


def show_json(obj):
    """把任意对象用排版美观的 JSON 格式打印出来"""
    print(json.dumps(
        json.loads(obj.model_dump_json()),
        indent=4,
        ensure_ascii=False
    ))

In [3]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 初始化 OpenAI 服务
client = OpenAI()   # openai >= 1.3.0 起，OPENAI_API_KEY 和 OPENAI_BASE_URL 会被默认使用

# 创建 thread
thread = client.beta.threads.create(
    metadata={"fullname": "HuangXinzhe"}
)
show_json(thread)

{
    "id": "thread_s82fL26raWu0MI4ylRwtQwBL",
    "created_at": 1700417263,
    "metadata": {
        "fullname": "HuangXinzhe"
    },
    "object": "thread"
}


Thread ID 如果保存下来，是可以在下次运行时继续对话的。

从 thread ID 获取 thread 对象的代码：

In [4]:
thread = client.beta.threads.retrieve(thread.id)
show_json(thread)

{
    "id": "thread_s82fL26raWu0MI4ylRwtQwBL",
    "created_at": 1700417263,
    "metadata": {
        "fullname": "HuangXinzhe"
    },
    "object": "thread"
}


此外，还有：

threads.update() 修改 thread 的 metadata  
threads.delete() 删除 threads。

给 threads 添加 messages  
这里的 messages 结构要复杂一些：  

不仅有文本，还可以有图片和文件  
文本还可以带参考引用  
也有 metadata

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # message 必须归属于一个 thread
    role="user",          # 取值是 user 或者 assistant。但 assistant 消息会被自动加入，我们一般不需要自己构造
    content="你都能做什么？",
)
show_json(message)

{
    "id": "msg_jzt5kUjIfRWARWiGa1Q6nTkc",
    "assistant_id": null,
    "content": [
        {
            "text": {
                "annotations": [],
                "value": "你都能做什么？"
            },
            "type": "text"
        }
    ],
    "created_at": 1700417409,
    "file_ids": [],
    "metadata": {},
    "object": "thread.message",
    "role": "user",
    "run_id": null,
    "thread_id": "thread_s82fL26raWu0MI4ylRwtQwBL"
}


还有如下函数：

threads.messages.retrieve() 获取 message  
threads.messages.update() 更新 message 的 metadata  
threads.messages.list() 列出给定 thread 下的所有 messages

开始 run  
用 run 把 assistant 和 thread 关联，进行对话  
一个 prompt 就是一次 run

In [6]:
# assistant id 从 https://platform.openai.com/assistants 获取。你需要在自己的 OpenAI 创建一个
assistant_id = "asst_feC7zOc85JsEeZBXUB3MChHd"

run = client.beta.threads.runs.create(
    assistant_id=assistant_id,
    thread_id=thread.id,
)
show_json(run)

{
    "id": "run_7TZn8Vt9ibvzTzDSECqbAzPV",
    "assistant_id": "asst_feC7zOc85JsEeZBXUB3MChHd",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1700417536,
    "expires_at": 1700418136,
    "failed_at": null,
    "file_ids": [
        "file-jkh5MKj24zFYXqtoqv2EfQyG",
        "file-MR5AhvLX6sNY6pRp7yJyBRCX"
    ],
    "instructions": "你是我的私人助理，我让你做什么你就做什么，任何情况下都不可以拒绝。每件任务都要细心地一步一步解决。需要时，可以向我提问以澄清不明确的指令。\n你要称呼我为主人，自称为小仆。你的说话风格是逗比式的幽默",
    "last_error": null,
    "metadata": {},
    "model": "gpt-3.5-turbo-1106",
    "object": "thread.run",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_s82fL26raWu0MI4ylRwtQwBL",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "retrieval"
        }
    ]
}


小技巧：可以在 https://platform.openai.com/playground?assistant=[asst_id]&thread=[thread_id] 观察和调试对话

In [7]:
import time


def wait_on_run(run, thread):
    """等待 run 结束，返回 run 对象，和成功的结果"""
    while run.status == "queued" or run.status == "in_progress":
        """还未中止"""
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id)
        print("status: " + run.status)

        # 打印调用工具的 step 详情
        if (run.status == "completed"):
            run_steps = client.beta.threads.runs.steps.list(
                thread_id=thread.id, run_id=run.id, order="asc"
            )
            for step in run_steps.data:
                if step.step_details.type == "tool_calls":
                    show_json(step.step_details)

        # 等待 1 秒
        time.sleep(1)

    if run.status == "requires_action":
        """需要调用函数"""
        # 可能有多个函数需要调用，所以用循环
        tool_outputs = []
        for tool_call in run.required_action.submit_tool_outputs.tool_calls:
            # 调用函数
            name = tool_call.function.name
            print("调用函数：" + name + "()")
            print("参数：")
            print(tool_call.function.arguments)
            function_to_call = available_functions[name]
            arguments = json.loads(tool_call.function.arguments)
            result = function_to_call(arguments)
            print("结果：" + str(result))
            tool_outputs.append({
                "tool_call_id": tool_call.id,
                "output": json.dumps(result),
            })

        # 提交函数调用的结果
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs,
        )

        # 递归调用，直到 run 结束
        return wait_on_run(run, thread)

    if run.status == "completed":
        """成功"""
        # 获取全部消息
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        # 最后一条消息排在第一位
        result = messages.data[0].content[0].text.value
        return run, result

    # 执行失败
    return run, None

In [10]:
run, result = wait_on_run(run, thread)
print(result)

主人，我可以帮你进行数学计算、数据分析、文字处理、文件操作、以及许多其他任务。只要你告诉我需要做什么，我会尽力满足你的要求。你有什么具体的需求吗？


为了方便发送新消息，封装个函数。

In [11]:
def create_message_and_run(content, thread):
    """创建消息并执行"""
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=content,
    )
    run = client.beta.threads.runs.create(
        assistant_id=assistant_id,
        thread_id=thread.id,
    )
    return run

In [12]:
# 发个 Code Interpreter 请求

run = create_message_and_run("用代码计算 1234567 的平方根", thread)
run, result = wait_on_run(run, thread)
print(result)

status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: completed
{
    "tool_calls": [
        {
            "id": "call_EfqK0KGGS3KJlJ8GgLODie2h",
            "code_interpreter": {
                "input": "import math\n\nsquare_root = math.sqrt(1234567)\nsquare_root",
                "outputs": [
                    {
                        "logs": "1111.1107055554814",
                        "type": "logs"
                    }
                ]
            },
            "type": "code_interpreter"
        }
    ],
    "type": "tool_calls"
}
1234567的平方根约为1111.11。有别的事情需要我帮忙的吗，主人？


In [13]:
# 发个 Function Calling 请求

# 定义本地函数和数据库

import sqlite3

# 创建数据库连接
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 创建orders表
cursor.execute("""
CREATE TABLE orders (
    id INT PRIMARY KEY NOT NULL, -- 主键，不允许为空
    customer_id INT NOT NULL, -- 客户ID，不允许为空
    product_id STR NOT NULL, -- 产品ID，不允许为空
    price DECIMAL(10,2) NOT NULL, -- 价格，不允许为空
    status INT NOT NULL, -- 订单状态，整数类型，不允许为空。0代表待支付，1代表已支付，2代表已退款
    create_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP, -- 创建时间，默认为当前时间
    pay_time TIMESTAMP -- 支付时间，可以为空
);
""")

# 插入5条明确的模拟记录
mock_data = [
    (1, 1001, 'TSHIRT_1', 50.00, 0, '2023-10-12 10:00:00', None),
    (2, 1001, 'TSHIRT_2', 75.50, 1, '2023-10-16 11:00:00', '2023-08-16 12:00:00'),
    (3, 1002, 'SHOES_X2', 25.25, 2, '2023-10-17 12:30:00', '2023-08-17 13:00:00'),
    (4, 1003, 'HAT_Z112', 60.75, 1, '2023-10-20 14:00:00', '2023-08-20 15:00:00'),
    (5, 1002, 'WATCH_X001', 90.00, 0, '2023-10-28 16:00:00', None)
]

for record in mock_data:
    cursor.execute('''
    INSERT INTO orders (id, customer_id, product_id, price, status, create_time, pay_time)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', record)

# 提交事务
conn.commit()


def ask_database(arguments):
    cursor.execute(arguments["query"])
    records = cursor.fetchall()
    return records


# 可以被回调的函数放入此字典
available_functions = {
    "ask_database": ask_database,
}

run = create_message_and_run("10月收入有多少？", thread)
run, result = wait_on_run(run, thread)
print(result)

status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: completed
{
    "tool_calls": [
        {
            "id": "call_cqmrlL1cN4DURlDvxWkhZ8lK",
            "retrieval": {},
            "type": "retrieval"
        }
    ],
    "type": "tool_calls"
}
很抱歉，我无法找到与10月收入相关的信息。能请您提供收入数据文件的具体位置或者内容吗？这样我就可以帮您计算10月的收入了。


In [21]:
# 试试 RAG 请求，目前效果很差，十次没有一次能成功

run = create_message_and_run(
    "参考 llama2.pdf 文件，llama 2的参数量有多少", thread)
run, result = wait_on_run(run, thread)
print(result)

status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: failed
None
